# Clusterizador com UNet

## Imports

In [ ]:
import torch
import torch.nn as nn

import torchvision.transforms.functional as TF

from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive

from os import listdir
from os.path import isfile, join

from tqdm import tqdm

import random

import torch.optim as optim

'''
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
from torch import sigmoid
from torch.nn.modules.upsampling import Upsample
from torch.nn.functional import interpolate
from torch.autograd import Variable
from torch.nn import MaxPool2d
from torch.nn.modules.conv import Conv2d
from torch.nn.modules.activation import Sigmoid, ReLU

from torchsummary import summary

from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from scipy.spatial.distance import cosine as cosine_distance
'''

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd

from os import listdir
from os.path import isfile, join

from tqdm import tqdm

import random

import torch
from torch import nn, sigmoid
from torch.nn.modules.upsampling import Upsample
from torch.nn.functional import interpolate
from torch.autograd import Variable
from torch.nn import MaxPool2d
from torch.nn.modules.conv import Conv2d
from torch.nn.modules.activation import Sigmoid, ReLU
# from Encoders import global_attention 

from torchsummary import summary

from google.colab import drive

from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import normalize
from scipy.spatial.distance import cosine as cosine_distance

from torchsummary import summary

import torchvision.transforms as transforms

## Data and Google Drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
pathCC = "/content/drive/MyDrive/Guilherme/csv_data/CC"
pathGL = "/content/drive/MyDrive/Guilherme/csv_data/GL"
pathLL = "/content/drive/MyDrive/Guilherme/csv_data/LL"

In [ ]:
mypath_CC = pathCC
CC_names = [f for f in listdir(mypath_CC) if isfile(join(mypath_CC, f))]

mypath_GL = pathGL
GL_names = [f for f in listdir(mypath_GL) if isfile(join(mypath_GL, f))]

mypath_LL = pathLL
LL_names = [f for f in listdir(mypath_LL) if isfile(join(mypath_LL, f))]

In [ ]:
dataset_CC, dataset_GL, dataset_LL = [], [], []
C, G, L = 0, 0, 0

for name in tqdm(CC_names):
    
    data = np.genfromtxt(mypath_CC + "/" + name, delimiter=',')
    data = data[1:]
    data = np.expand_dims(data, axis=0)
    
    dataset_CC.append(data)

    C += 1

    if C == 1460: break

for name in tqdm(GL_names):
    
    data = np.genfromtxt(mypath_GL + "/" + name, delimiter=',')
    data = data[1:]
    data = np.expand_dims(data, axis=0)
    
    dataset_GL.append(data)

    G += 1

    if G == 1460: break

for name in tqdm(LL_names):
    
    data = np.genfromtxt(mypath_LL + "/" + name, delimiter=',')
    data = data[1:]
    data = np.expand_dims(data, axis=0)
    
    dataset_LL.append(data)

    L += 1

    if L == 1460: break

 98%|█████████▊| 1459/1488 [17:13<00:20,  1.41it/s]


In [ ]:
random.shuffle(dataset_CC)
random.shuffle(dataset_GL)
random.shuffle(dataset_LL)

In [ ]:
train_set, valid_set = [], []

for counter, value in enumerate(dataset_CC):
    
    if counter < 1200:
        imagem = (value, 0)
        train_set.append(imagem)
    else:
        imagem = (value, 0)
        valid_set.append(imagem)

for counter, value in enumerate(dataset_GL):
    
    if counter < 1200:
        imagem = [value, 1]
        train_set.append(imagem)
    else:
        imagem = [value, 1]
        valid_set.append(imagem)

for counter, value in enumerate(dataset_LL):
    
    if counter < 1200:
        imagem = [value, 2]
        train_set.append(imagem)
    else:
        imagem = [value, 2]
        valid_set.append(imagem)

In [ ]:
classes = ['Curto circuito', 'Perda de Geração', 'Perda de Carga']
# classes = ['Perda de Geração', 'Perda de Carga']

In [ ]:
def imshow(img):
    img = img / 2 + 0.5  
    plt.imshow(np.transpose(img, (0,1))) 

## Utils

In [ ]:
class CustomDataset():
    
    def __init__(self, dataset, transform=None):
        
        self.images    = []
        self.targets   = []
        self.transform = transform

        for i in range(len(dataset)):

            self.images.append(dataset[i][0])
            self.targets.append(dataset[i][1])
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        
        image  = self.images[index] 
        target = self.targets[index] 
        
        if self.transform:
            image = self.transform(image)
            
        return (image, index, target)

In [ ]:
def cluster_test(dataset, model):

    valid_loader = torch.utils.data.DataLoader(dataset, batch_size=8, num_workers=0)
    o, l = [], []
    lab, out = [], []

    for inputs in valid_loader:        

        images, labels = inputs
        images = images.to(device)

        output = model(images.float())
        output = output.to("cpu")

        output = output.detach().numpy()
        labels = labels.numpy()

        o.append(output)
        l.append(labels)    

    for i in range(len(l)):
        for j in range(len(o[i])):
            lab.append(l[i][j])
            out.append(o[i][j])

    kmeans = KMeans(n_clusters=3, init='k-means++', n_init=10, max_iter=300, tol=0.01, verbose=0, random_state=None, copy_x=True, algorithm='elkan')
    label = kmeans.fit_predict(out)

    i00, i01, i02 = 0, 0, 0
    i10, i11, i12 = 0, 0, 0
    i20, i21, i22 = 0, 0, 0

    for i in range(len(label)):

        if lab[i] == 0 and label[i] == 0:
            i00 += 1
        if lab[i] == 0 and label[i] == 1:
            i01 += 1
        if lab[i] == 0 and label[i] == 2:
            i02 += 1

        if lab[i] == 1 and label[i] == 0:
            i10 += 1
        if lab[i] == 1 and label[i] == 1:
            i11 += 1
        if lab[i] == 1 and label[i] == 2:
            i12 += 1

        if lab[i] == 2 and label[i] == 0:
            i20 += 1
        if lab[i] == 2 and label[i] == 1:
            i21 += 1
        if lab[i] == 2 and label[i] == 2:
            i22 += 1

    p = [[i00, i01, i02],
        [i10, i11, i12],
        [i20, i21, i22]]

    print(p)
    print()

## Model

### Double Conv

In [ ]:
# Define model of double convolution

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        
        self.conv = nn.Sequential(
        
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        
        )
        
    def forward(self, x):
        return self.conv(x)

In [ ]:
x     = torch.randn((1, 1, 160, 160))
model = DoubleConv(1, 3)
preds = model(x)

print(x.shape)
print(preds.shape)

torch.Size([1, 1, 160, 160])
torch.Size([1, 3, 160, 160])


### Encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, in_channels, features=[64, 128, 256, 512]):
        super(Encoder, self).__init__()
        
        self.downs = nn.ModuleList()
        self.pool  = nn.MaxPool2d(kernel_size=2, stride=2)
        
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature
            
        
        
    def forward(self, x):

        self.skip_connections = []
        
        for down in self.downs:
            x = down(x)
            self.skip_connections.append(x)
            x = self.pool(x)
        
        return x

In [ ]:
x     = torch.randn((1, 1, 160, 160))
model = Encoder(1)
preds = model(x)

print(x.shape)
print(preds.shape)
print(f'\nConnections:\n')
for i in model.skip_connections:
    print(i.shape)

torch.Size([1, 1, 160, 160])
torch.Size([1, 512, 10, 10])

Connections:

torch.Size([1, 64, 160, 160])
torch.Size([1, 128, 80, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 512, 20, 20])


### UNet

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels, features):
        super(UNet, self).__init__()
        
        self.encoder = Encoder(in_channels, features)
        self.ups     = nn.ModuleList()
        
        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(feature*2, feature, kernel_size=2, stride=2))
            self.ups.append(DoubleConv(feature*2, feature))
            
        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)
        
        
        
    def forward(self, x):

        self.decoder_skip_connections = []
        
        x = self.encoder(x)
        
        x = self.bottleneck(x)
        
        self.decoder_skip_connections.append(x)

        
        self.encoder_skip_connections = self.encoder.skip_connections[::-1]
        
        for i in range(0, len(self.ups), 2):
            
            x = self.ups[i](x)
            
            self.decoder_skip_connections.append(x)
        
            skip_connection = self.encoder_skip_connections[i//2]
            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])
                
            concat_skip = torch.cat((skip_connection, x), dim=1)
            
            x = self.ups[i+1](concat_skip)
        
        return self.final_conv(x)

In [ ]:
x     = torch.randn((1, 1, 360, 68))
model = UNet(1, 1, [64, 128, 256, 512])
preds = model(x)

print(x.shape)
print(preds.shape)

print(f'\nConnections:\n')
for i in reversed(model.encoder_skip_connections):
    print(i.shape)

print(f'')    

for i, m in enumerate(model.decoder_skip_connections):
    print(m.shape)
    if i == 0:
        print(f'')
        
summary(model, input_size=(1, 360, 68))

torch.Size([1, 1, 360, 68])
torch.Size([1, 1, 360, 68])

Connections:

torch.Size([1, 64, 360, 68])
torch.Size([1, 128, 180, 34])
torch.Size([1, 256, 90, 17])
torch.Size([1, 512, 45, 8])

torch.Size([1, 1024, 22, 4])

torch.Size([1, 512, 44, 8])
torch.Size([1, 256, 90, 16])
torch.Size([1, 128, 180, 34])
torch.Size([1, 64, 360, 68])


RuntimeError: ignored

In [ ]:
encoder = model.encoder

x     = torch.randn((1, 1, 360, 68))
preds = encoder(x)

print(x.shape)
print(preds.shape)

### Encoder Modificado

In [ ]:
class EncoderMerged(Encoder):
    def __init__(self, in_channels, features):
        super(EncoderMerged, self).__init__(in_channels, features)

    def forward(self, x):

        x = super().forward(x)
        x_code = torch.mean(x, dim = (-2, -1))

        return x_code

In [ ]:
x     = torch.randn((1, 1, 160, 160))
model = EncoderMerged(1, [64, 128, 256, 512])
preds = model(x)

print(x.shape)
print(preds.shape)
print(f'\nConnections:\n')
for i in model.skip_connections:
    print(i.shape)

torch.Size([1, 1, 160, 160])
torch.Size([1, 512])

Connections:

torch.Size([1, 64, 160, 160])
torch.Size([1, 128, 80, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 512, 20, 20])


### UNet com Encoder Modificado

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels, features):
        super(UNet, self).__init__()
        
        self.encoder = EncoderMerged(in_channels, features)
        self.ups     = nn.ModuleList()
        
        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(feature*2, feature, kernel_size=2, stride=2))
            self.ups.append(DoubleConv(feature*2, feature))
            
        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)
        
        self.decoder_skip_connections = []
        
    def forward(self, x):
        
        x = self.encoder(x)
        
        x = self.bottleneck(x)
        
        self.decoder_skip_connections.append(x)

        
        self.encoder_skip_connections = self.encoder.skip_connections[::-1]
        
        for i in range(0, len(self.ups), 2):
            
            x = self.ups[i](x)
            
            self.decoder_skip_connections.append(x)
        
            skip_connection = self.encoder_skip_connections[i//2]
            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])
                
            concat_skip = torch.cat((skip_connection, x), dim=1)
            
            x = self.ups[i+1](concat_skip)
        
        return self.final_conv(x)

In [ ]:
model = UNet(1, 1, [64, 128, 256, 512])

encoder = model.encoder

x     = torch.randn((1, 1, 360, 68))
preds = encoder(x)

print(x.shape)
print(preds.shape)

## VGG Model

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
       
        # ENCODER =============================
        
        self.encoder = nn.Sequential(
            
            Conv2d(1, 64, 3, 1, 1, bias=True),
            nn.BatchNorm2d(64),
            ReLU(),
            Conv2d(64, 64, 3, 1, 1, bias=True),
            nn.BatchNorm2d(64),
            ReLU(),
            nn.MaxPool2d(2, 2),
                
            Conv2d(64, 128, 3, 1, 1, bias=True),
            nn.BatchNorm2d(128),
            ReLU(),
            Conv2d(128, 128, 3, 1, 1, bias=True),
            nn.BatchNorm2d(128),
            ReLU(),
            nn.MaxPool2d(2, 2),
                
            Conv2d(128, 256, 3, 1, 1, bias=True),
            nn.BatchNorm2d(256),
            ReLU(),
            Conv2d(256, 256, 3, 1, 1, bias=True),
            nn.BatchNorm2d(256),
            ReLU(),
            Conv2d(256, 256, 3, 1, 1, bias=True),
            nn.BatchNorm2d(256),
            ReLU(),
            nn.MaxPool2d(2, 2),

            Conv2d(256, 512, 3, 1, 1, bias=True),
            nn.BatchNorm2d(512),
            ReLU(),
            Conv2d(512, 512, 3, 1, 1, bias=True),
            nn.BatchNorm2d(512),
            ReLU(),
            Conv2d(512, 512, 3, 1, 1, bias=True),
            nn.BatchNorm2d(512),
            ReLU(),
            nn.MaxPool2d(2, 2)
            
        )

        
    def forward(self, x):
      
        x = self.encoder(x)
              
        return x

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
         
        # DECODER =============================

        self.decoder = nn.Sequential(
        
            nn.ConvTranspose2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(256),
            ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),
            nn.BatchNorm2d(128),
            ReLU(),

            nn.ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(64),
            ReLU(),
            nn.ConvTranspose2d(64, 64, kernel_size=(10, 10), stride=(2, 2), padding=(1, 1)),
            nn.BatchNorm2d(64),
            ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

        )

        
    def forward(self, x):
        
        x = self.decoder(x)
        
        x = torch.sigmoid(x)
              
        return x

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = Encoder()
        self.decoder = Decoder()
        
    def forward(self, x):

        x = self.encoder(x)
        
        x = self.decoder(x)
        
        x = torch.sigmoid(x)
              
        return x

In [ ]:
class EncoderMerged(Encoder):
    def __init__(self):
        super(EncoderMerged, self).__init__()

    def forward(self, x):

        x = super().forward(x)
        x_code = torch.mean(x, dim = (-2, -1))

        return x_code

In [ ]:
class AutoEncoderMerged(nn.Module):
    def __init__(self):
        super(AutoEncoderMerged, self).__init__()

        self.encoder = EncoderMerged()
        self.decoder = Decoder()
        
    def forward(self, x):

        x = self.encoder(x)
        
        x = self.decoder(x)
        
        x = torch.sigmoid(x)
              
        return x

## Treino

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device

#Instantiate the model
model = UNet(1, 1, [64, 128, 256, 512])

device = get_device()
# print(device)
model = model.to(device)

In [ ]:
args = {'batch_size'          : 8,
        'lr'                  : 0.01,
        'weight_decay'        : 0.0,
        'n_epochs'            : 200,
        'momentum'            : 0.9,
        'scheduler_step_size' : 15,
        'scheduler_gamma'     : 0.1
        }

In [ ]:
ts = CustomDataset(train_set)
vs = CustomDataset(valid_set)

train_loader = torch.utils.data.DataLoader(ts, batch_size=args['batch_size'], num_workers=0, shuffle=True)
valid_loader = torch.utils.data.DataLoader(vs, batch_size=args['batch_size'], num_workers=0)

In [ ]:
criterion = nn.L1Loss().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay']) # 5h33

In [ ]:
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'], weight_decay=args['weight_decay'])

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'], weight_decay=args['weight_decay'])

In [ ]:
n_epochs = args['n_epochs']
loss_per_epoch = []    

for epoch in range(1, n_epochs+1):
      
    # monitor training loss
    train_loss = 0.0

    #Training
    for data in train_loader:

            
        images, _, _ = data
        images    = images.to(device).float()
        optimizer.zero_grad()  
        
        # Forward

        outputs = model(images)
        loss = criterion(outputs.float(), images.float())

        #Backward
        
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*images.size(0)

    train_loss = train_loss/len(train_loader)
    loss_per_epoch.append(train_loss)      


    if epoch % 10 == 0:
        print('Epoch: {} \tTraining Loss: {:.6f} <- SAVE'.format(epoch, train_loss))
        torch.save(model.state_dict(), '/content/drive/MyDrive/Guilherme/MODEL_SAVE_v17/conv_autoencoder_' + str(epoch) + '.pth')
    else:
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

Epoch: 1 	Training Loss: 0.000000
Epoch: 2 	Training Loss: 0.000000
Epoch: 3 	Training Loss: 0.000000
Epoch: 4 	Training Loss: 0.000000
Epoch: 5 	Training Loss: 0.000000
Epoch: 6 	Training Loss: 0.000000
Epoch: 7 	Training Loss: 0.000000
Epoch: 8 	Training Loss: 0.000000
Epoch: 9 	Training Loss: 0.000000
Epoch: 10 	Training Loss: 0.000000 <- SAVE
Epoch: 11 	Training Loss: 0.000000
Epoch: 12 	Training Loss: 0.000000
Epoch: 13 	Training Loss: 0.000000
Epoch: 14 	Training Loss: 0.000000
Epoch: 15 	Training Loss: 0.000000
Epoch: 16 	Training Loss: 0.000000
Epoch: 17 	Training Loss: 0.000000
Epoch: 18 	Training Loss: 0.000000
Epoch: 19 	Training Loss: 0.000000
Epoch: 20 	Training Loss: 0.000000 <- SAVE
Epoch: 21 	Training Loss: 0.000000
Epoch: 22 	Training Loss: 0.000000
Epoch: 23 	Training Loss: 0.000000
Epoch: 24 	Training Loss: 0.000000
Epoch: 25 	Training Loss: 0.000000
Epoch: 26 	Training Loss: 0.000000
Epoch: 27 	Training Loss: 0.000000
Epoch: 28 	Training Loss: 0.000000
Epoch: 29 	Tr

In [ ]:
plt.plot(loss_per_epoch)

## Load

In [ ]:
def cluster_test(dataset, model):

    valid_loader = torch.utils.data.DataLoader(dataset, batch_size=8, num_workers=0)
    o, l = [], []
    lab, out = [], []

    for inputs in valid_loader:        

        images, labels = inputs
        images = images.to(device)

        output = model.encoder(images.float())
        output = output.to("cpu")

        output = output.detach().numpy()
        labels = labels.numpy()

        o.append(output)
        l.append(labels)    

    for i in range(len(l)):
        for j in range(len(o[i])):
            lab.append(l[i][j])
            out.append(o[i][j])

    kmeans = KMeans(n_clusters=3, init='k-means++', n_init=10, max_iter=300, tol=0.01, verbose=0, random_state=None, copy_x=True, algorithm='elkan')
    label = kmeans.fit_predict(out)

    i00, i01, i02 = 0, 0, 0
    i10, i11, i12 = 0, 0, 0
    i20, i21, i22 = 0, 0, 0

    for i in range(len(label)):

        if lab[i] == 0 and label[i] == 0:
            i00 += 1
        if lab[i] == 0 and label[i] == 1:
            i01 += 1
        if lab[i] == 0 and label[i] == 2:
            i02 += 1

        if lab[i] == 1 and label[i] == 0:
            i10 += 1
        if lab[i] == 1 and label[i] == 1:
            i11 += 1
        if lab[i] == 1 and label[i] == 2:
            i12 += 1

        if lab[i] == 2 and label[i] == 0:
            i20 += 1
        if lab[i] == 2 and label[i] == 1:
            i21 += 1
        if lab[i] == 2 and label[i] == 2:
            i22 += 1

    p = [[i00, i01, i02],
        [i10, i11, i12],
        [i20, i21, i22]]

    print(p)
    print()

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device

#Instantiate the model
model = UNet(1, 1, [64, 128, 256, 512])
model.load_state_dict(torch.load('/content/drive/MyDrive/Guilherme/MODEL_SAVE_v17/conv_autoencoder_200.pth'))
'''
del model.ups
del model.bottleneck
del model.final_conv
del model.encoder.downs[3].conv[5]
del model.encoder.downs[3].conv[4]
'''
# model.eval()

device = get_device()
print(device)
model.to(device)

cuda:0


UNet(
  (encoder): EncoderMerged(
    (downs): ModuleList(
      (0): DoubleConv(
        (conv): Sequential(
          (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU(inplace=True)
        )
      )
      (1): DoubleConv(
        (conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, af

In [ ]:
cluster_test(train_set, model)
# [[883, 1, 316], [414, 411, 375], [1188, 0, 12]]
# [[887, 313, 0], [467, 376, 357], [1116, 84, 0]]

[[372, 112, 716], [165, 162, 873], [125, 4, 1071]]



## Cluster

### func

In [ ]:
def marsaglia(sphere_dim):
    '''Method to generate a point uniformly distributed on the (N-1) sphere by Marsaglia
    Args:
        sphere_dim (int): dimension of the sphere on which to generate the point
    '''
    norm_vals = np.random.standard_normal(sphere_dim)
    return norm_vals / np.linalg.norm(norm_vals)

In [ ]:
class MemoryBank(object):

    def __init__(self, n_vectors, dim_vector, memory_mixing_rate=None):

        self.dim_vector = dim_vector
        self.vectors = np.array([marsaglia(dim_vector) for _ in range(n_vectors)])
        self.memory_mixing_rate = memory_mixing_rate
        self.mask_init = np.array([False] * n_vectors)

    def update_memory(self, vectors, index):

        if isinstance(index, int):
            self.vectors[index] = self._update_(vectors, self.vectors[index])

        elif isinstance(index, np.ndarray):
            for ind, vector in zip(index, vectors):
                # print(ind)
                # print(vector)
                self.vectors[ind] = self._update_(vector, self.vectors[ind])

        else:
            raise RuntimeError('Index must be of type integer or NumPy array, not {}'.format(type(index)))

    def mask(self, inds_int):
        ret_mask = []
        for row in inds_int:
            row_mask = np.full(self.vectors.shape[0], False)
            row_mask[row.astype(int)] = True
            ret_mask.append(row_mask)

        return np.array(ret_mask)

    def _update_(self, vector_new, vector_recall):
        v_add = vector_new * self.memory_mixing_rate + vector_recall * (1.0 - self.memory_mixing_rate)
        return v_add / np.linalg.norm(v_add)

    def _verify_dim_(self, vector_new):
        if len(vector_new) != self.dim_vector:
            raise VectorUpdateError('Update vector of dimension size {}, '.format(len(vector_new)) + \
                                    'but memory of dimension size {}'.format(self.dim_vector))

In [ ]:
class LocalAggregationLoss(nn.Module):

    def __init__(self, temperature,
                 k_nearest_neighbours, clustering_repeats, number_of_centroids,
                 memory_bank,
                 kmeans_n_init=1, nn_metric=cosine_distance, nn_metric_params={},
                 include_self_index=True, force_stacking=False):
        super(LocalAggregationLoss, self).__init__()

        self.temperature = temperature
        self.memory_bank = memory_bank
        self.include_self_index = include_self_index
        self.force_stacking = force_stacking

        self.background_neighbours = None
        self.close_neighbours = None

        self.neighbour_finder = NearestNeighbors(n_neighbors=k_nearest_neighbours + 1,
                                                 algorithm='ball_tree',
                                                 metric=nn_metric, metric_params=nn_metric_params)
        self.clusterer = []
        for k_clusterer in range(clustering_repeats):

            self.clusterer.append(KMeans(n_clusters=number_of_centroids, init='random', n_init=kmeans_n_init))

    def _nearest_neighbours(self, codes_data, indices):

        self.neighbour_finder.fit(self.memory_bank.vectors)
        indices_nearest = self.neighbour_finder.kneighbors(codes_data, return_distance=False)

        if not self.include_self_index:
            self_neighbour_masks = [np.where(indices_nearest[k] == indices[k]) for k in range(indices_nearest.shape[0])]
            if any([len(x) != 1 for x in self_neighbour_masks]):
                raise RuntimeError('Self neighbours not correctly shaped')
            indices_nearest = np.delete(indices_nearest, self_neighbour_masks, axis=1)

        return self.memory_bank.mask(indices_nearest)

    def _close_grouper(self, indices):
      
        memberships = [[]] * len(indices)
        for clusterer in self.clusterer:
            clusterer.fit(self.memory_bank.vectors)
            
            for k_index, cluster_index in enumerate(clusterer.labels_[indices]):
                other_members = np.where(clusterer.labels_ == cluster_index)[0]
                other_members_union = np.union1d(memberships[k_index], other_members)
                memberships[k_index] = other_members_union.astype(int)

        return self.memory_bank.mask(np.array(memberships, dtype=object))

    def _intersecter(self, n1, n2):
      
        ret = [[v1 and v2 for v1, v2 in zip(n1_x, n2_x)] for n1_x, n2_x in zip(n1, n2)]
        return np.array(ret)

    def _prob_density(self, codes, indices, force_stack=False):
      
        ragged = len(set([np.count_nonzero(ind) for ind in indices])) != 1

        if not ragged and not force_stack:
            vals = torch.tensor([np.compress(ind, self.memory_bank.vectors, axis=0) for ind in indices],
                                requires_grad=False)
            v_dots = torch.matmul(vals, codes.unsqueeze(-1))
            exp_values = torch.exp(torch.div(v_dots, self.temperature))
            xx = torch.sum(exp_values, dim=1).squeeze(-1)

        else:
            xx_container = []
            for k_item in range(codes.size(0)):
                vals = torch.tensor(np.compress(indices[k_item], self.memory_bank.vectors, axis=0),
                                    requires_grad=False)
                v_dots_prime = torch.mv(vals, codes[k_item])
                exp_values_prime = torch.exp(torch.div(v_dots_prime, self.temperature))
                xx_prime = torch.sum(exp_values_prime, dim=0)
                xx_container.append(xx_prime)
            xx = torch.stack(xx_container, dim=0)

        return xx

    def forward(self, codes, indices):

        assert codes.shape[0] == len(indices)

        # print(indices)

        codes = codes.type(torch.DoubleTensor)
        code_data = normalize(codes.detach().numpy(), axis=1)

        self.memory_bank.update_memory(code_data, indices)
        self.background_neighbours = self._nearest_neighbours(code_data, indices)
        self.close_neighbours = self._close_grouper(indices)
        self.neighbour_intersect = self._intersecter(self.background_neighbours, self.close_neighbours)

        v = F.normalize(codes, p=2, dim=1)
        d1 = self._prob_density(v, self.background_neighbours, self.force_stacking)
        d2 = self._prob_density(v, self.neighbour_intersect, self.force_stacking)
        loss_cluster = torch.sum(torch.log(d1) - torch.log(d2)) / codes.shape[0]

        return loss_cluster

### train

In [ ]:
args = {'batch_size'   : 40,
        'lr'           : 0.005,
        'weight_decay' : 0.0,
        'n_epochs'     : 100,
        'n_epochs_re'  : 20
        }

In [ ]:
ts = CustomDataset(train_set)
vs = CustomDataset(valid_set)

train_loader = torch.utils.data.DataLoader(ts, batch_size=args['batch_size'], num_workers=0, shuffle=True)
valid_loader = torch.utils.data.DataLoader(vs, batch_size=args['batch_size'], num_workers=0)

In [ ]:
memory_bank = MemoryBank(n_vectors=len(ts), dim_vector=512, memory_mixing_rate=0.5) # 0.5

criterion = LocalAggregationLoss(memory_bank          = memory_bank,
                                 temperature          = 0.05,   # 0.07
                                 k_nearest_neighbours = 500,    # 500
                                 clustering_repeats   = 6,      # 6
                                 number_of_centroids  = 90)     # 3

optimizer = optim.SGD(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

ep = args['n_epochs_re']

for epoch in range(args['n_epochs_re']):
    print(f'Epoch: {epoch+1} : {ep}')

    for inputs in train_loader:        

        images, idx, _ = inputs

        images = images.to(device)

        idx = idx.cpu().numpy()

        optimizer.zero_grad()
        output = model.encoder(images.float())

        loss = criterion(output.float(), idx)
        loss.backward()
        optimizer.step()

    cluster_test(train_set, model)

Epoch: 1 : 20


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


[[686, 0, 514], [405, 768, 27], [1119, 0, 81]]

Epoch: 2 : 20
[[0, 665, 535], [767, 406, 27], [0, 1119, 81]]

Epoch: 3 : 20
[[664, 0, 536], [405, 768, 27], [1119, 0, 81]]

Epoch: 4 : 20
[[670, 0, 530], [406, 767, 27], [1120, 0, 80]]

Epoch: 5 : 20
[[0, 536, 664], [767, 27, 406], [0, 82, 1118]]

Epoch: 6 : 20
[[1200, 0, 0], [419, 211, 570], [1200, 0, 0]]

Epoch: 7 : 20
[[1200, 0, 0], [416, 220, 564], [1200, 0, 0]]

Epoch: 8 : 20
[[1200, 0, 0], [416, 221, 563], [1200, 0, 0]]

Epoch: 9 : 20
[[1200, 0, 0], [417, 580, 203], [1200, 0, 0]]

Epoch: 10 : 20
[[1200, 0, 0], [419, 575, 206], [1200, 0, 0]]

Epoch: 11 : 20
[[1200, 0, 0], [419, 575, 206], [1200, 0, 0]]

Epoch: 12 : 20
[[1200, 0, 0], [419, 206, 575], [1200, 0, 0]]

Epoch: 13 : 20
[[1200, 0, 0], [419, 213, 568], [1200, 0, 0]]

Epoch: 14 : 20
[[1200, 0, 0], [416, 571, 213], [1200, 0, 0]]

Epoch: 15 : 20


In [ ]:
memory_bank = MemoryBank(n_vectors=len(ts), dim_vector=512, memory_mixing_rate=0.5) # 0.5

criterion = LocalAggregationLoss(memory_bank          = memory_bank,
                                 temperature          = 0.07,   # 0.07
                                 k_nearest_neighbours = 500,    # 500
                                 clustering_repeats   = 6,      # 6
                                 number_of_centroids  = 100)    # 3

optimizer = optim.SGD(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

ep = args['n_epochs_re']

for epoch in range(args['n_epochs_re']):
    print(f'Epoch: {epoch+1} : {ep}')

    for inputs in valid_loader:        

        images, idx, _ = inputs

        images = images.to(device)

        idx = idx.cpu().numpy()

        optimizer.zero_grad()
        output = model.encoder(images.float())

        loss = criterion(output.float(), idx)
        loss.backward()
        optimizer.step()

    cluster_test(train_set, model)

Epoch: 1 : 20
[[0, 828, 372], [769, 427, 4], [0, 1193, 7]]

Epoch: 2 : 20
[[590, 182, 428], [631, 137, 432], [1126, 12, 62]]

Epoch: 3 : 20
[[0, 681, 519], [771, 401, 28], [0, 1131, 69]]

Epoch: 4 : 20
[[688, 0, 512], [399, 773, 28], [1130, 0, 70]]

Epoch: 5 : 20
[[716, 0, 484], [398, 774, 28], [1132, 0, 68]]

Epoch: 6 : 20
[[709, 0, 491], [398, 774, 28], [1131, 0, 69]]

Epoch: 7 : 20
[[0, 510, 690], [771, 28, 401], [0, 71, 1129]]

Epoch: 8 : 20
[[729, 0, 471], [405, 767, 28], [1128, 0, 72]]

Epoch: 9 : 20
[[724, 0, 476], [405, 767, 28], [1129, 0, 71]]

Epoch: 10 : 20
[[475, 0, 725], [28, 767, 405], [70, 0, 1130]]

Epoch: 11 : 20
[[474, 0, 726], [28, 767, 405], [71, 0, 1129]]

Epoch: 12 : 20


KeyboardInterrupt: ignored

Epoch: 1 : 20
[[183, 0, 77], [12, 246, 2], [260, 0, 0]]

Epoch: 2 : 20
[[77, 183, 0], [2, 13, 245], [0, 260, 0]]

Epoch: 3 : 20
[[0, 183, 77], [243, 15, 2], [0, 260, 0]]

Epoch: 4 : 20
[[183, 0, 77], [16, 241, 3], [260, 0, 0]]

Epoch: 5 : 20
[[161, 0, 99], [16, 241, 3], [260, 0, 0]]

Epoch: 6 : 20
[[143, 0, 117], [13, 244, 3], [260, 0, 0]]

Epoch: 7 : 20
[[142, 0, 118], [12, 245, 3], [260, 0, 0]]

Epoch: 8 : 20
[[136, 0, 124], [11, 244, 5], [260, 0, 0]]

Epoch: 9 : 20
[[136, 0, 124], [11, 244, 5], [260, 0, 0]]

Epoch: 10 : 20
[[131, 0, 129], [11, 244, 5], [260, 0, 0]]

Epoch: 11 : 20
[[0, 131, 129], [243, 12, 5], [0, 260, 0]]

Epoch: 12 : 20
[[143, 117, 0], [5, 14, 241], [0, 260, 0]]

Epoch: 13 : 20
[[120, 140, 0], [15, 5, 240], [260, 0, 0]]

Epoch: 14 : 20
[[120, 140, 0], [16, 5, 239], [260, 0, 0]]

Epoch: 15 : 20
[[140, 0, 120], [5, 239, 16], [0, 0, 260]]

Epoch: 16 : 20
[[0, 127, 133], [239, 16, 5], [0, 260, 0]]

In [ ]:
ts = CustomDataset(train_set)
vs = CustomDataset(valid_set)

train_loader = torch.utils.data.DataLoader(ts, batch_size=args['batch_size'], num_workers=0, shuffle=True)
valid_loader = torch.utils.data.DataLoader(vs, batch_size=args['batch_size'], num_workers=0)

for i in range(10):

    def get_device():
      if torch.cuda.is_available():
          device = 'cuda:0'
      else:
          device = 'cpu'
      return device

    #Instantiate the model
    model = UNet(1, 1, [32, 64, 128, 256, 512])
    path = '/content/drive/MyDrive/Guilherme/MODEL_SAVE_v12/conv_autoencoder_' + str((i+1)*20) + '.pth'
    model.load_state_dict(torch.load(path))
    model.eval()

    device = get_device()
    model  = model.to(device)

    cluster_test(train_set, model)

[[497, 134, 569], [447, 201, 552], [1165, 0, 35]]

[[583, 135, 482], [771, 181, 248], [1189, 0, 11]]

[[462, 44, 694], [162, 150, 888], [11, 0, 1189]]

[[643, 557, 0], [917, 17, 266], [1165, 35, 0]]

[[255, 629, 316], [248, 895, 57], [14, 1184, 2]]

[[1130, 70, 0], [1199, 0, 1], [1200, 0, 0]]



/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1255: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_


[[1200, 0, 0], [1200, 0, 0], [1200, 0, 0]]



/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1255: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_


[[1200, 0, 0], [1200, 0, 0], [1200, 0, 0]]



/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1255: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_


[[1200, 0, 0], [1200, 0, 0], [1200, 0, 0]]

[[1200, 0, 0], [1200, 0, 0], [1200, 0, 0]]



/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1255: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_


In [ ]:
ts = CustomDataset(train_set)
vs = CustomDataset(valid_set)

train_loader = torch.utils.data.DataLoader(ts, batch_size=args['batch_size'], num_workers=0, shuffle=True)
valid_loader = torch.utils.data.DataLoader(vs, batch_size=args['batch_size'], num_workers=0)

for i in range(10):

    def get_device():
      if torch.cuda.is_available():
          device = 'cuda:0'
      else:
          device = 'cpu'
      return device

    #Instantiate the model
    model = UNet(1, 1, [32, 64, 128, 256, 512])
    path = '/content/drive/MyDrive/Guilherme/MODEL_SAVE_v13/conv_autoencoder_' + str((i+1)*20) + '.pth'
    model.load_state_dict(torch.load(path))
    model.eval()

    device = get_device()
    model  = model.to(device)

    cluster_test(train_set, model)

# [[445, 0, 755], [413, 770, 17], [1164, 0, 36]]
# [[732, 468, 0], [10, 420, 770], [27, 1173, 0]]
# [[0, 675, 525], [194, 498, 508], [0, 1194, 6]]

[[516, 0, 684], [419, 766, 15], [1163, 0, 37]]

[[481, 0, 719], [414, 770, 16], [1158, 0, 42]]

[[718, 0, 482], [14, 770, 416], [35, 0, 1165]]

[[469, 731, 0], [412, 19, 769], [1154, 46, 0]]

[[688, 512, 0], [15, 415, 770], [37, 1163, 0]]

[[528, 0, 672], [416, 767, 17], [1156, 0, 44]]

[[543, 0, 657], [418, 769, 13], [1169, 0, 31]]

[[660, 0, 540], [418, 647, 135], [1194, 0, 6]]

[[549, 651, 0], [92, 417, 691], [11, 1189, 0]]

[[665, 535, 0], [416, 145, 639], [1194, 6, 0]]



In [ ]:
ts = CustomDataset(train_set)
vs = CustomDataset(valid_set)

train_loader = torch.utils.data.DataLoader(ts, batch_size=args['batch_size'], num_workers=0, shuffle=True)
valid_loader = torch.utils.data.DataLoader(vs, batch_size=args['batch_size'], num_workers=0)

for i in range(20):

    def get_device():
      if torch.cuda.is_available():
          device = 'cuda:0'
      else:
          device = 'cpu'
      return device

    #Instantiate the model
    model = UNet(1, 1, [32, 64, 128, 256, 512])
    path = '/content/drive/MyDrive/Guilherme/MODEL_SAVE_v14/conv_autoencoder_' + str((i+1)*10) + '.pth'
    model.load_state_dict(torch.load(path))
    model.eval()

    device = get_device()
    model  = model.to(device)

    cluster_test(train_set, model)

[[736, 464, 0], [439, 133, 628], [1188, 12, 0]]

[[501, 0, 699], [136, 626, 438], [16, 0, 1184]]

[[684, 516, 0], [453, 101, 646], [1179, 21, 0]]

[[0, 679, 521], [683, 445, 72], [0, 1176, 24]]

[[0, 686, 514], [666, 453, 81], [0, 1178, 22]]

[[0, 682, 518], [673, 454, 73], [0, 1178, 22]]

[[509, 691, 0], [80, 454, 666], [22, 1178, 0]]

[[0, 681, 519], [687, 454, 59], [0, 1178, 22]]

[[682, 518, 0], [454, 59, 687], [1178, 22, 0]]

[[0, 504, 696], [666, 78, 456], [0, 19, 1181]]

[[704, 496, 0], [470, 59, 671], [1183, 17, 0]]

[[752, 448, 0], [465, 75, 660], [1189, 11, 0]]

[[687, 0, 513], [465, 673, 62], [1178, 0, 22]]

[[750, 450, 0], [474, 61, 665], [1188, 12, 0]]

[[492, 708, 0], [58, 460, 682], [16, 1184, 0]]

[[0, 747, 453], [680, 464, 56], [0, 1188, 12]]

[[690, 510, 0], [459, 60, 681], [1181, 19, 0]]

[[509, 691, 0], [58, 466, 676], [19, 1181, 0]]

[[674, 0, 526], [451, 699, 50], [1178, 0, 22]]

[[725, 0, 475], [460, 684, 56], [1187, 0, 13]]



In [ ]:
cluster_test(valid_set, model)
# [[209, 51, 0], [101, 31, 128], [260, 0, 0]]
# [[209, 0, 51], [101, 29, 130], [260, 0, 0]]
# [[110, 46, 104], [202, 8, 50], [251, 0, 9]]
# [[0, 0, 0], [175, 85, 0], [0, 260, 0]]
# [[0, 0, 0], [144, 116, 0], [0, 260, 0]]

[[0, 0, 0], [149, 111, 0], [0, 260, 0]]



In [ ]:
cluster_test(train_set, model)

ValueError: ignored

In [ ]:
def cluster_test(dataset, model):

    valid_loader = torch.utils.data.DataLoader(dataset, batch_size=8, num_workers=0)
    o, l = [], []
    lab, out = [], []

    for inputs in valid_loader:        

        images, labels = inputs
        images = images.to(device)

        output = model.encoder(images.float())
        output = output.to("cpu")

        output = output.detach().numpy()
        labels = labels.numpy()

        o.append(output)
        l.append(labels)    

    for i in range(len(l)):
        for j in range(len(o[i])):
            lab.append(l[i][j])
            out.append(o[i][j])

    kmeans = KMeans(n_clusters=3, n_init=10, max_iter=300)
    label = kmeans.fit_predict(out)

    i00, i01, i02 = 0, 0, 0
    i10, i11, i12 = 0, 0, 0
    i20, i21, i22 = 0, 0, 0

    for i in range(len(label)):

        if lab[i] == 0 and label[i] == 0:
            i00 += 1
        if lab[i] == 0 and label[i] == 1:
            i01 += 1
        if lab[i] == 0 and label[i] == 2:
            i02 += 1

        if lab[i] == 1 and label[i] == 0:
            i10 += 1
        if lab[i] == 1 and label[i] == 1:
            i11 += 1
        if lab[i] == 1 and label[i] == 2:
            i12 += 1

        if lab[i] == 2 and label[i] == 0:
            i20 += 1
        if lab[i] == 2 and label[i] == 1:
            i21 += 1
        if lab[i] == 2 and label[i] == 2:
            i22 += 1

    p = [[i00, i01, i02],
        [i10, i11, i12],
        [i20, i21, i22]]

    print(p)
    print()

In [ ]:
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=8, num_workers=0)

o, l = [], []

for inputs in valid_loader:        

    images, labels = inputs

    images = images.to(device)

    output = model.encoder(images.float())
    output = output.to("cpu")

    output = output.detach().numpy()
    labels = labels.numpy()

    o.append(output)
    l.append(labels)

In [ ]:
output.shape

(4, 512)

In [ ]:
len(l)

98

In [ ]:
lab, out = [], []

for i in range(98):
    for j in range(len(o[i])):
        lab.append(l[i][j])
        out.append(o[i][j])

len(out)

780

In [ ]:
kmeans = KMeans(n_clusters=3, n_init=10, max_iter=300)
label = kmeans.fit_predict(out)

In [ ]:
i00, i01, i02 = 0, 0, 0
i10, i11, i12 = 0, 0, 0
i20, i21, i22 = 0, 0, 0

for i in range(780):

    if lab[i] == 0 and label[i] == 0:
        i00 += 1
    if lab[i] == 0 and label[i] == 1:
        i01 += 1
    if lab[i] == 0 and label[i] == 2:
        i02 += 1

    if lab[i] == 1 and label[i] == 0:
        i10 += 1
    if lab[i] == 1 and label[i] == 1:
        i11 += 1
    if lab[i] == 1 and label[i] == 2:
        i12 += 1

    if lab[i] == 2 and label[i] == 0:
        i20 += 1
    if lab[i] == 2 and label[i] == 1:
        i21 += 1
    if lab[i] == 2 and label[i] == 2:
        i22 += 1

In [ ]:
p = [[i00, i01, i02],
     [i10, i11, i12],
     [i20, i21, i22]]

In [ ]:
print(p)

[[148, 0, 112], [19, 237, 4], [260, 0, 0]]


## Resto

In [ ]:
#Batch of test images
dataiter = iter(valid_loader)
images, labels = dataiter.next()

images = images.to(device)

#Sample outputs
output = model.encoder(images.float())
output = output.to("cpu")

print(output.shape)

'''
images = images.to("cpu")
images = images.numpy()

output = output.view(args['batch_size'], 1, 68, 360)
output = output.to("cpu")
output = output.detach().numpy()
'''

In [ ]:
#Batch of test images
dataiter = iter(valid_loader)
images, labels = dataiter.next()

images = images.to(device)

#Sample outputs
output = model(images.float())

images = images.to("cpu")
images = images.numpy()

output = output.view(args['batch_size'], 1, 68, 360)
output = output.to("cpu")
output = output.detach().numpy()

k = 5
#Original Images
print("Original Images")
fig = plt.figure(figsize=(15,7))
for idx in np.arange(k, k+2):
    ax = fig.add_subplot(1, 2, idx+1-k, xticks=[], yticks=[])
    imshow(images[idx].squeeze())
    ax.set_title(classes[labels[idx]])
plt.show()

#Reconstructed Images
print('Reconstructed Images')
fig = plt.figure(figsize=(15,7))
for idx in np.arange(k, k+2):
    ax = fig.add_subplot(1, 2, idx+1-k, xticks=[], yticks=[])
    imshow(output[idx].squeeze())
    ax.set_title(classes[labels[idx]])
plt.show()

In [ ]:
plt.plot(np.transpose(images[4].squeeze(), (1, 0)))
plt.show()

In [ ]:
plt.plot(np.transpose(output[4].squeeze(), (1, 0)))
plt.show()

In [ ]:
images[7].squeeze().shape

In [ ]:
def imshow(img):
    img = img / 2 + 0.5  
    plt.imshow(np.transpose(img, (0,1))) 

In [ ]:
for i in range(0, 68):
  plt.plot(np.transpose(images[4].squeeze(), (0, 1))[i])
  plt.plot(np.transpose(output[4].squeeze(), (0, 1))[i])
  plt.show()

In [ ]:
v = np.transpose(output[4].squeeze(), (0, 1))
v.shape